# Grab Safety Analysis

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Configure data visualization
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# Background and Objective
Safety is an important aspect for online transportation. We want customers to feel safe riding Grab so that they could do other things on the way without worry. 

Customers could feel unsafe because of driver's behaviour or driving skill, eg: 
- Driver using unpopular shortcuts 
- Driver talk with another person on the phone or with customers
- Driver keep seeing GPS and don't pay attention to the road
- Sleepy 
- Speeding
- Harsh acceleration, braking, or cornering
- Run over speed bump/hole with high speed

If we could quickly detect when the driver starts driving unsafely, we could remind the driver real-time to prevent something bad happen.  

# Raw Data

In [ ]:
import glob

def read_csv_from_folder(path_pattern:str, file_limit:int=None):
    """Read csv files from path pattern. Only read file_limit files if it is defined"""
    file_names = glob.glob(path_pattern)
    file_content_list = []
    
    limit = len(file_names)
    if file_limit is not None:
        limit=min(file_limit, limit)
        
    print('Reading {} files from "{}" ...'.format(limit, path_pattern))
    
    for file_name in file_names[:limit]:
        df = pd.read_csv(file_name, index_col=None, header=0)
        file_content_list.append(df)
    
    contents = pd.concat(file_content_list, axis=0, ignore_index=True)
    del file_content_list
    return contents

features_raw = read_csv_from_folder('../data/features/*.csv', file_limit=2)
labels = read_csv_from_folder('../data/labels/*.csv')

In [ ]:
print('Features data sample:')
features_raw.head()
print('Labels data sample:')
labels.head()

In [ ]:
features_raw.loc[features_raw.bookingID==1202590843006,:].sort_values(by='second').head()

## Feature description
### Booking id
- Trip id
- Possibly relate to service type (GrabCar/Bike) ?

### Accuracy
- Accuracy inferred by GPS in meters
- Affect uncertainty level for GPS Bearing and Speed

### Bearing
- GPS bearing in degree
- The degree of the GPS movement relative from North
- Could relate with GPS accuracy. Less accurate means more uncertainty in the real speed.
- Beware that 10 degree to 340 degrees is 30 degrees difference

### Acceleration (x, y, z)
- Accelerometer reading at a certain axis (m/s2)
- [Youtube explaination about how Accelerometer works](https://www.youtube.com/watch?v=KZVgKu6v808)
- Concern: how could we factor out gravity acceleration.
- Concern: phone orientation

### Gyro (x, y, z)
- Gyroscope reading in certain axis (rad/s)
- Measure angular velocity / speed of rotation
- [Explaination about how Gyroscope works](https://learn.sparkfun.com/tutorials/gyroscope/all)
- Concern: Gyroscope bias, usually caused by heat

### Second
- Time of the record by number of seconds
- Remember that it is not in constant interval, eg: per 2s. If we want to use lag, we need to add for time delta or interpolate it.

### Speed
- Speed measured by GPS in m/s
- Could relate with GPS accuracy. Less accurate means more uncertainty in the real speed.

# Basic Data Preprocess

### Split to train and test set by bookingID
We only split the data to become two sets. Train and test set and we use the test set as the validation set as well. The data is split based on bookingID and before preprocessing to make sure there is no data leak from train to test set.

In [ ]:
train_dataset_ratio = 0.7

all_booking_ids = features_raw.bookingID.unique()
np.random.seed(1)
train_booking_id = np.random.choice(all_booking_ids, 
                                    size = int(train_dataset_ratio * all_booking_ids.shape[0]), 
                                    replace=False)

train_dataset = features_raw.loc[features_raw.bookingID.isin(train_booking_id), :].copy(deep=False)
train_label = labels.loc[labels.bookingID.isin(train_booking_id), :].copy(deep=False)
test_dataset = features_raw.loc[~features_raw.bookingID.isin(train_booking_id), :].copy(deep=False)
test_label = labels.loc[~labels.bookingID.isin(train_booking_id), :].copy(deep=False)

In [ ]:
print('Safe and un-save trips')
labels.label.value_counts()
print('\n')
print('#BookingID with more than 1 rows in labels: {0}'.format((labels.bookingID.value_counts() > 1).sum()))

### Handle double label

In [ ]:
def preproces_label(labels):
    return labels.groupby(['bookingID']).max().reset_index().copy(deep=False)

In [ ]:
train_label = preproces_label(train_label)
test_label = preproces_label(test_label)

# Feature Engineering

### Sequence and Sort

In [ ]:
def ensure_sorted(dataset):
    dataset_copy = dataset.copy(deep=False)
    
    dataset_copy['sequence'] = dataset_copy[
        ['bookingID', 'second']
    ].groupby('bookingID').rank(ascending=True, method='first')

    dataset_copy = dataset_copy.sort_values(by=['bookingID', 'second'])
    return dataset_copy

### Gyroscope Data

- Usually phone is not rotating all the time and the value of gyroscope will be 0.
- [Knowing that there is a bias of gyroscope reading](https://base.xsens.com/hc/en-us/articles/209611089-Understanding-Sensor-Bias-offset-), we could use mean to find the expected reading while the phone is in stable position. 

In [ ]:
sns.distplot(features_raw.loc[features_raw.bookingID==1477468749954,['gyro_x']])
print('Gyroscope reading, x-axis bias:', features_raw.loc[features_raw.bookingID==1477468749954,['gyro_x']].mean());

In [ ]:
def gyro_data_enrich(dataset):
    enriched_dataset = dataset.copy(deep=False)
    enriched_dataset = ensure_sorted(enriched_dataset)
    
    gyro_cols = ['gyro_x', 'gyro_y', 'gyro_z']
    
    # Find gyroscope bias / stable values
    for col in gyro_cols:
        if (col+'_stable') in enriched_dataset.columns:
            continue
        agg_stable = enriched_dataset.groupby('bookingID')[col].mean().reset_index()
        agg_stable.columns = ['bookingID', col+'_stable']
        enriched_dataset = pd.merge(enriched_dataset, agg_stable, how='left', on='bookingID', validate='m:1', copy=False)

    # Gyroscope filtered / calibrated values
    for col in gyro_cols:
        if (col+'_filtered') in enriched_dataset.columns:
            continue
        enriched_dataset[col+'_filtered'] = enriched_dataset[col] - enriched_dataset[col+'_stable']
    
    # Gyroscope magnitude of calibrated values
    enriched_dataset['gyro_filtered_magnitude'] = np.sqrt(enriched_dataset['gyro_x_filtered']**2 + \
                                                          enriched_dataset['gyro_y_filtered']**2 + \
                                                          enriched_dataset['gyro_z_filtered']**2)
    
    # Gyroscope magnitude standard deviation
    agg_std = enriched_dataset.groupby('bookingID')['gyro_filtered_magnitude'].std().reset_index()
    agg_std.columns = ['bookingID', 'gyro_filtered_std']
    enriched_dataset = pd.merge(enriched_dataset, agg_std, how='left', on='bookingID', validate='m:1', copy=False)
        
    return enriched_dataset

In [ ]:
# gyro_data_enrich(train_dataset).head()

### Accelerometer Data

- Accelerometer readings depends on gravity
- [Phone orientation](https://www.digikey.com/en/articles/techzone/2011/may/using-an-accelerometer-for-inclination-sensing) could change over time and change the gravity acceleration for each axis

Could we?
- Handle accelerometer bias? Gravity is not always 9.8. It depends on altitude and accelerometer bias. 
- Distinguish between vehicle movement and user moving the phone?
- Normalize all data assuming all phones are having the same orientation?

In [ ]:
def accel_data_enrich(dataset, smoothing: int=3):
    enriched_dataset = dataset.copy(deep=False)
    enriched_dataset = ensure_sorted(enriched_dataset)
    
    accel_cols = pd.Series(['acceleration_x', 'acceleration_y', 'acceleration_z'])
    
    # Rolling mean of accleration data to find gravity
    rolling_mean_data = enriched_dataset.groupby('bookingID').apply(
        lambda x: x[
            accel_cols
        ].rolling(window=smoothing, min_periods=1, center=True).mean())
    rolling_mean_data.columns = accel_cols + '_gravity'
    enriched_dataset = pd.concat([enriched_dataset, rolling_mean_data], axis=1, verify_integrity=True)
    
    # Acceleration magnitude
    enriched_dataset['acceleration_magnitude'] = np.sqrt(enriched_dataset['acceleration_x']**2 + \
                                                         enriched_dataset['acceleration_y']**2 + \
                                                         enriched_dataset['acceleration_z']**2) 
    
    # Current acceleration vs gravity diff
    for col in accel_cols:
        enriched_dataset[col+'_gravity_diff'] = enriched_dataset[col] - enriched_dataset[col+'_gravity']
    enriched_dataset['acceleration_gravity_diff_magnitude'] = np.sqrt(enriched_dataset['acceleration_x_gravity_diff']**2 + \
                                                                      enriched_dataset['acceleration_y_gravity_diff']**2 + \
                                                                      enriched_dataset['acceleration_z_gravity_diff']**2) 
    
    # Acceleration magnitude standard deviation
    agg_std = enriched_dataset.groupby('bookingID')['acceleration_magnitude', 'acceleration_gravity_diff_magnitude'].std().reset_index()
    agg_std.columns = ['bookingID', 'acceleration_std', 'acceleration_gravity_diff_std']
    enriched_dataset = pd.merge(enriched_dataset, agg_std, how='left', on='bookingID', validate='m:1', copy=False)
    
    # Phone orientation
    enriched_dataset['orientation_theta'] = np.arctan(enriched_dataset.acceleration_x_gravity / \
        np.sqrt(enriched_dataset.acceleration_y_gravity**2 + enriched_dataset.acceleration_z_gravity**2)) / np.pi * 360
    enriched_dataset['orientation_psi'] = np.arctan(enriched_dataset.acceleration_y_gravity / \
        np.sqrt(enriched_dataset.acceleration_x_gravity**2 + enriched_dataset.acceleration_z_gravity**2)) / np.pi * 360
    enriched_dataset['orientation_phi'] = np.arctan( np.sqrt(enriched_dataset.acceleration_x_gravity**2 + enriched_dataset.acceleration_y_gravity**2) / \
        enriched_dataset.acceleration_z_gravity ) / np.pi * 360
    
    return enriched_dataset

In [ ]:
# accel_data_enrich(train_dataset).head()

### Sequence Difference Data and Aggregation

In [ ]:
def diff_data_enrich(dataset):
    enriched_dataset = dataset.copy(deep=False)
    enriched_dataset = ensure_sorted(enriched_dataset)
    
    # Construct diff
    diff_data = enriched_dataset.groupby('bookingID')['second','Bearing','Speed'].diff()
    diff_data = diff_data.rename(columns = lambda x: x + '_diff')
    
    # Modify Bearing diff to -180 to 180 
    diff_data.Bearing_diff = diff_data.Bearing_diff
    diff_data.Bearing_diff[diff_data.Bearing_diff < -180.0] += 180
    diff_data.Bearing_diff[diff_data.Bearing_diff > 180.0] -= 180

    # Difference / second (normalization)
    diff_data['Bearing_dps'] = diff_data['Bearing_diff'] / diff_data['second_diff']
    diff_data['Speed_dps'] = diff_data['Speed_diff'] / diff_data['second_diff']
    
    # Combine
    diff_data = diff_data.fillna(0)
    enriched_dataset = pd.concat([enriched_dataset, diff_data], axis=1, verify_integrity=True)
    
    # Combine accuracy of two sequence
    acc_sum = enriched_dataset.groupby('bookingID')['Accuracy']\
       .rolling(window=2, min_periods=1).sum().reset_index(drop=True).tolist()
    enriched_dataset['Accuracy_sum'] = acc_sum
    
    return enriched_dataset

In [ ]:
# diff_data_enrich(train_dataset)

In [ ]:
def aggregate_data(preprocessed_dataset):
    features_max = ['gyro_filtered_magnitude',
                    'acceleration_magnitude',
                    'Speed',
                    'Bearing_dps',
                    'Speed_dps',
                    'Accuracy_sum',
                    'second',
                    'sequence',
                    'acceleration_x_gravity_diff',
                    'acceleration_y_gravity_diff',
                    'acceleration_z_gravity_diff',
                    'acceleration_gravity_diff_magnitude',
                    'gyro_x_filtered',
                    'gyro_y_filtered',
                    'gyro_z_filtered']
    
    agg_max = preprocessed_dataset.groupby('bookingID')[features_max].max().reset_index()
    agg_max.columns = ['bookingID'] +  (pd.Series(features_max) + '_max').tolist()

    features_std = ['gyro_filtered_magnitude', 'acceleration_gravity_diff_magnitude']
    agg_std = preprocessed_dataset.groupby('bookingID')[features_std].std().reset_index()
    agg_std.columns = ['bookingID'] +  (pd.Series(features_std) + '_std').tolist()
    
    agg_data = pd.merge(agg_max, agg_std, on='bookingID', validate='1:1')
    agg_data['second_sequence_ratio'] = agg_data['second_max'] / agg_data['sequence_max'].astype(float)
    return agg_data

In [ ]:
def preprocess(dataset):
    dataset = gyro_data_enrich(dataset)
    dataset = accel_data_enrich(dataset, smoothing=5)
    dataset = diff_data_enrich(dataset)
    return dataset

# Analysis

In [ ]:
train_dataset_prep = preprocess(train_dataset)

## Charting

In [ ]:
def chart_trip(dataset, booking_id):
    booking_id_data = dataset.loc[dataset.bookingID==booking_id,:].sort_values(by='second')
    
    plt.figure(figsize=(15,10))
    plt.subplots_adjust(hspace = .001)
    
    # Acceleration
    booking_id_acc = booking_id_data[
        ['second','acceleration_x', 'acceleration_y','acceleration_z', 'acceleration_magnitude']
    ].melt(id_vars=["second"], var_name="axis", value_name="value")
    ax1 = plt.subplot('311')
    
    plt.title("Measured data for booking ID: {}".format(booking_id))
    sns.lineplot(x="second", y="value", hue='axis', data=booking_id_acc, ax=ax1, marker="o");
    
    # Gyroscope
    booking_id_gyro = booking_id_data[
        ['second', 'gyro_x', 'gyro_y', 'gyro_z']
    ].melt(id_vars=["second"], var_name="axis", value_name="value")
    ax2 = plt.subplot('312')
    sns.lineplot(x="second", y="value", hue='axis', data=booking_id_gyro, ax=ax2, markers=True, marker="o");
    
    # Speed
    booking_id_speed = booking_id_data[
        ['second', 'Speed', 'Accuracy']
    ].melt(id_vars=["second"], var_name="type", value_name="value")
    ax3 = plt.subplot('313')
    sns.lineplot(x='second', y="value", hue='type', data=booking_id_speed, ax=ax3, markers=True, marker="o")

In [ ]:
chart_trip(train_dataset_prep, 1477468749954)

### Sample of non-safe trip

In [ ]:
samp = train_label.bookingID[train_label.label == 1].sample(5, random_state=2)
for id in samp:
    chart_trip(train_dataset_prep, id)

### Sample of safe trip

In [ ]:
samp = train_label.bookingID[train_label.label == 0].sample(5, random_state=2)
for id in samp:
    chart_trip(train_dataset_prep, id)

## Correlation

In [ ]:
analytics_features = ['gyro_filtered_magnitude',
                      'acceleration_magnitude',
                      'Speed',
                      'Bearing_dps',
                      'Speed_dps',
                      'second_diff',
                      'second',
                      'Accuracy_sum',
                      'acceleration_x', 
                      'acceleration_y', 
                      'acceleration_z',
                      'acceleration_x_gravity_diff',
                      'acceleration_y_gravity_diff',
                      'acceleration_z_gravity_diff',
                      'acceleration_gravity_diff_magnitude',
                      'gyro_x_filtered',
                      'gyro_y_filtered',
                      'gyro_z_filtered']

### Mean correlation

In [ ]:
analytics_mean_corr = train_dataset_prep.groupby('bookingID')[analytics_features].mean().reset_index()
analytics_mean_corr = pd.merge(analytics_mean_corr, train_label, on='bookingID')
analytics_mean_corr = analytics_mean_corr.corr()

plt.figure( figsize=(15,5) )
ax1 = plt.subplot2grid((1, 4), (0, 0), colspan=2)
ax2 = plt.subplot2grid((1, 4), (0, 3), colspan=1)
sns.heatmap(analytics_mean_corr, ax=ax1)
sns.heatmap(pd.DataFrame(analytics_mean_corr.loc[analytics_mean_corr.index != 'label','label']), annot=True, ax=ax2);

This correlation table shows that mean/average data is less effective to determine a trip is save or unsafe. The unsafe tracking event maybe only recorded 1 time or maybe < 5% of the trip. But, acceleration and gyroscope magnitude data stand out here. How many % of the trips where the driver consistently drive unsafely?

### Max correlation

In [ ]:
analytics_max_corr = train_dataset_prep.groupby('bookingID')[analytics_features].max().reset_index()
analytics_max_corr = pd.merge(analytics_max_corr, train_label, on='bookingID')
analytics_max_corr = analytics_max_corr.corr()

plt.figure( figsize=(15,5) )
ax1 = plt.subplot2grid((1, 4), (0, 0), colspan=2)
ax2 = plt.subplot2grid((1, 4), (0, 3), colspan=1)
sns.heatmap(analytics_max_corr, ax=ax1)
sns.heatmap(pd.DataFrame(analytics_max_corr.loc[analytics_max_corr.index != 'label','label']), annot=True, ax=ax2);

### Standard deviation correlation

In [ ]:
analytics_std_corr = train_dataset_prep.groupby('bookingID')[analytics_features].std().reset_index()
analytics_std_corr = pd.merge(analytics_std_corr, train_label, on='bookingID')
analytics_std_corr = analytics_std_corr.corr()

plt.figure( figsize=(15,5) )
ax1 = plt.subplot2grid((1, 4), (0, 0), colspan=2)
ax2 = plt.subplot2grid((1, 4), (0, 3), colspan=1)
sns.heatmap(analytics_std_corr, ax=ax1)
sns.heatmap(pd.DataFrame(analytics_std_corr.loc[analytics_std_corr.index != 'label','label']), annot=True, ax=ax2);

Gyro and acceleration standard deviation is highly correlated. The hypothesis is when there is angular velocity (gyroscope reading), the phone orientation is changing. Hence, the direction of gravity relative to the phone is changing and the reading for each accelerometer axis is changing too. 

Because it is highly correlated, we will just use subset of it:
- gyroscope filter magnitude's standard deviation
- acceleration gravity diff magnitude's standard deviation

## Others

### Could we cluster the trips?

In [ ]:
train_agg_data = aggregate_data(train_dataset_prep)
train_agg_data = pd.merge(train_agg_data, train_label, on='bookingID', validate='1:1')

In [ ]:
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
features = train_agg_data.columns[train_agg_data.columns.str.contains("max|std")]

std_scaler = preprocessing.StandardScaler()
x = std_scaler.fit_transform(train_agg_data[features])

pca = PCA(n_components=2)
pc = pca.fit_transform(x)
pc_df = pd.DataFrame(data = pc, columns = ['pc1', 'pc2'])

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(x="pc1", 
                y="pc2", 
                hue="label",
                data=pd.concat([train_agg_data, pc_df], axis=1, verify_integrity=True).sample(2000));

There are **no visually standout clusters** after we reduce the features to 2-dimension with PCA. 

### Does trip duration effects safety?
There are two possibilities (hypothesis)
1. Longer trips increase the probability to encounter non-safe events
2. There are trips with a duration of < 2 minutes. Probably short duration trips are cancelled trips and that is why most of them are safe trips.

In [ ]:
sns.distplot(np.log2(train_agg_data.loc[(train_agg_data.label == 0) & (train_agg_data.second_max < 7200)].second_max), 
             label='safe', color='green')
sns.distplot(np.log2(train_agg_data.loc[(train_agg_data.label == 1) & (train_agg_data.second_max < 7200)].second_max), 
             label='not safe', color='red')
plt.xlim(xmax=15)
plt.xlabel('log2(trip duration in second)')
plt.legend();

In [ ]:
bins =  [i for i in range(0,(40+1),2)] # List of 0, 2,4,6 .., 40
df_pct_area = train_agg_data.loc[train_agg_data.second_max < (40*60), ['bookingID','second_max', 'label']]
df_pct_area['duration_min'] = df_pct_area.second_max / 60.0
df_pct_area['2min_bin'] = pd.cut(df_pct_area['duration_min'], bins=bins, labels=bins[:-1])
df_pct_area = pd.pivot_table(df_pct_area, values='bookingID', aggfunc=lambda x: x.shape[0], index='2min_bin', columns='label')
df_pct_area = df_pct_area.divide(df_pct_area.sum(axis=1), axis=0)

In [ ]:
# Make the plot
plt.stackplot(range(0,40,2),  df_pct_area[1],  df_pct_area[0], labels=['not safe','safe'], 
              colors=['#83d0c9','#ff6f69'])
plt.legend(loc='upper left')
plt.margins(0,0)
plt.title('Percentage Area Chart of Trip Safety')
plt.xlabel('duration (min)')
plt.show();

After visualizing this chart, because of the un-safe trip proportion increase linearly, we believe more on the first hypothesis over the other one. Longer trips increase the probability to encounter non-safe event. This belief is strengthened when we see the other side which is almost all trips after 40 minutes are non-safe trips.

Additionally, after seeing the smoothness of the chart, this data seems like a little bit odd. Seeing the pattern, there is a hypothesis that the label is produced by other machine learning model which have an additive effect. The model predicts each segment of the trip and adds up the unsafeness. I think using trip duration feature to train and predict is like some kind of "cheat" and we should not use it in the real case. 

### Does tracking rate have corellation with label?

In [ ]:
sns.distplot(train_agg_data.loc[(train_agg_data.label == 0) & (train_agg_data.second_sequence_ratio < 40.0)].second_sequence_ratio, 
             label='safe', color='green')
sns.distplot(train_agg_data.loc[(train_agg_data.label == 1) & (train_agg_data.second_sequence_ratio < 40.0)].second_sequence_ratio, 
             label='not safe', color='red')
plt.xlabel('Distribution of duration between sequence')
plt.legend();

There is almost no difference in event logging rate between labels.

# Modeling
Our goal is to make the model which is capable to find the pattern of non-safe event like:
- Driver using unpopular shortcuts 
- Driver talk with other person in phone or with customers
- Driver keep seeing GPS and don't pay attention to the road. 
- Sleepy 
- Speeding
- Harsh acceleration, braking, or cornering
- Run over speed bump / hole with high speed.

There are 4 rough ideas to approach this problem:
1. Learning on the summary of a trip
2. Stacking two models. The first model to detect non-safe events. The second model to summarize it.
3. We could also use CNN instead of stacking manually. We only need to learn patterns from sensor events which are close to each other. CNN fits the purpose.

## Evaluation

In [ ]:
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve

def combine_pred_label(prediction_df, label_df):
    """Combine two DataFrame, each DataFrame should contains 'bookingID' column."""
    return pd.merge(prediction_df, label_df, how='left', on='bookingID', validate='1:1')

def plot_roc(prediction_df, label_df):
    """Return ROC plot given prediction and label DataFrame. Both should have 'bookingID' column."""
    pred_label_df = combine_pred_label(prediction_df, label_df)
    
    fpr, tpr, thresholds = roc_curve(pred_label_df.label, pred_label_df.prediction)
    roc_auc = auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.3f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
def evaluate(prediction_df, label_df):
    """Return AUC evaluation given prediction and label DataFrame. Both should have 'bookingID' column."""
    pred_label_df = combine_pred_label(prediction_df, label_df)
    return roc_auc_score(pred_label_df.label, pred_label_df.prediction)
                         
def generate_second_dataset(dataset, prediciton, n_column=5):
    """
    Generate per booking secondary dataset from per event unsafeness prediction.
    Return top n_columns unsafeness for each bookingID
    
    Parameters:
    ----------
    dataset -- DataFrame contains bookingID
    prediction -- List / Series with length equal to dataset # rows. Each indicates unsafeness. 
    n_column -- number of column generated
    """
    sec_data = pd.DataFrame(data={'bookingID':dataset.bookingID, 'row_prob': prediciton})
    sec_data['rank'] = sec_data.groupby('bookingID').rank(ascending=False, method='first')
    sec_data = sec_data.loc[sec_data['rank'] <= n_column, :]
    sec_data = pd.pivot_table(data=sec_data, 
                              values='row_prob', 
                              index='bookingID', 
                              columns='rank', 
                              fill_value=0).reset_index()
    sec_data.columns=['bookingID'] + ['val_' + str(i) for i in range(1, (n_column + 1))]
    return sec_data

## 0. Baseline: Random

In [ ]:
import random

prediction_df = pd.DataFrame({
    'bookingID': test_dataset.bookingID.unique(),
    'prediction': np.random.random(size=test_dataset.bookingID.unique().shape[0])
})

In [ ]:
evaluate(prediction_df, test_label)
plot_roc(prediction_df, test_label)

## 1. Random Forest - Aggregated Data

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_dataset_prep = preprocess(train_dataset)

In [ ]:
train_agg_data = aggregate_data(train_dataset_prep)
train_agg_data = pd.merge(train_agg_data, train_label, on='bookingID', validate='1:1')

test_dataset_prep = preprocess(test_dataset)
test_agg_data = aggregate_data(test_dataset_prep)

features = train_agg_data.columns[train_agg_data.columns.str.contains("max|std|ratio")]

In [ ]:
cls = RandomForestClassifier(n_estimators=200, random_state=0, min_samples_leaf=75)
cls.fit(train_agg_data[features], train_agg_data.label)
pred = cls.predict_proba(test_agg_data[features])
pred = pred[:,np.argwhere(cls.classes_==1)[0][0]]
prediction_df = pd.DataFrame(data={'bookingID':test_agg_data.bookingID, 'prediction': pred})
print('AUC:',evaluate(prediction_df, test_label))
plot_roc(prediction_df, test_label);

In [ ]:
sns.distplot(pred);

## 2. LightGBM - Logistic Stack

In [ ]:
import lightgbm as lgb

In [ ]:
train_dataset_prep = preprocess(train_dataset)
train_dataset_prep = pd.merge(train_dataset_prep, train_label, on='bookingID', validate='m:1')
test_dataset_prep = preprocess(test_dataset)
test_dataset_prep = pd.merge(test_dataset_prep, test_label, on='bookingID', validate='m:1')

In [ ]:
train_dataset_prep.columns

In [ ]:
features = ['Accuracy', 'gyro_x_filtered', 'gyro_y_filtered', 'gyro_z_filtered', 'Speed', 'gyro_filtered_magnitude', 
            'acceleration_magnitude', 'Bearing_dps', 'Speed_dps', 'Accuracy_sum', 
            'acceleration_x_gravity_diff', 'acceleration_y_gravity_diff', 'acceleration_z_gravity_diff', 
            'acceleration_gravity_diff_magnitude', 'orientation_theta', 
            'orientation_psi', 'orientation_phi', 'second', 'acceleration_gravity_diff_std',
            'acceleration_std','gyro_filtered_std']


In [ ]:
cls = lgb.LGBMClassifier(boosting_type='dart', 
                         objective='binary', 
                         max_depth=6, 
                         n_estimator=100,
                         learning_rate=0.1, 
                         max_bin=100, 
                         num_leaves=70, 
                         lambda_l1=0.4,
                         min_data_in_leaf=150,
                         metric='auc')

cls.fit(train_dataset_prep[features], train_dataset_prep.label)

In [ ]:
# Comment this
y_pred = cls.predict_proba(test_dataset_prep[features])
y_pred = y_pred[:,np.argwhere(cls.classes_==1)[0][0]]
roc_auc_score(test_dataset_prep.label, y_pred)

### Stacking

In [ ]:
train_pred_first = cls.predict_proba(train_dataset_prep[features])
train_pred_first = train_pred_first[:,np.argwhere(cls.classes_==1)[0][0]]

test_pred_first = cls.predict_proba(test_dataset_prep[features])
test_pred_first = test_pred_first[:,np.argwhere(cls.classes_==1)[0][0]]

In [ ]:
from sklearn.linear_model import LogisticRegression

train_sec = generate_second_dataset(train_dataset_prep, train_pred_first, n_column=5)
train_sec = pd.merge(train_sec, train_label, on='bookingID')
sec_features = train_sec.columns[train_sec.columns.str.contains('val')]
sec_reg = LogisticRegression(random_state=0, penalty='l2', C=0.1)
sec_reg.fit(train_sec[sec_features], train_sec.label)

test_sec = generate_second_dataset(test_dataset_prep, test_pred_first, n_column=5)
test_sec = pd.merge(test_sec, test_label, on='bookingID')
y_pred = sec_reg.predict_proba(test_sec[sec_features])
y_pred = y_pred[:,np.argwhere(cls.classes_==1)[0][0]]
prediction_df = pd.DataFrame(data={'bookingID':test_sec.bookingID, 'prediction': y_pred})

evaluate(prediction_df, test_label)

In [ ]:
plot_roc(prediction_df, test_label)

In [ ]:
sns.distplot(prediction_df.prediction)

## 3. CNN
We will use [1D CNN for time series data / text](https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf)

In [ ]:
sns.distplot(train_agg_data.sequence_max)

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, BatchNormalization, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Concatenate, AveragePooling1D
from keras.preprocessing.sequence import pad_sequences 
from keras.regularizers import l1

def to_keras_input(dataset, features, maxlen=200) -> (list, pd.DataFrame):
    dataset_seq = dataset[
        list(set(['bookingID','second'] + features))
    ].groupby('bookingID').apply(
        lambda x: x.sort_values(by='second')[features].values.tolist()
    )
    booking_ids = pd.DataFrame({'idx': range(len(dataset_seq.index)) ,'bookingID':dataset_seq.index})
    dataset_seq = pad_sequences(dataset_seq, maxlen=maxlen, padding='pre', dtype=float, truncating='pre', value=0.0)
    return dataset_seq, booking_ids

def create_model_cnn1(dataset):
    num_seq = len(dataset[0])
    num_features = len(dataset[0][0])
    
    model_m = Sequential()
    model_m.add(Conv1D(16, 3, activation='relu', input_shape=(num_seq, num_features)))
    model_m.add(MaxPooling1D(3))
    model_m.add(Conv1D(32, 3, activation='relu', input_shape=(num_seq, num_features)))
    model_m.add(MaxPooling1D(3))
    model_m.add(Conv1D(64, 3, activation='relu', input_shape=(num_seq, num_features)))
    model_m.add(MaxPooling1D(3))
    model_m.add(GlobalAveragePooling1D())
    model_m.add(Dropout(0.2))
    model_m.add(Dense(16, activation='sigmoid'))
    model_m.add(Dense(1, activation='sigmoid'))
    print(model_m.summary())
    return model_m

def create_model_cnn_2(dataset):
    num_seq = len(dataset[0])
    num_features = len(dataset[0][0])
    
    inpt = Input(shape=(num_seq, num_features))
    
    convs = []
    
    conv1 = Conv1D(8, 1, activation='relu')(inpt)
    pool1 = GlobalMaxPooling1D()(conv1)
    convs.append(pool1)
    
    conv2 = Conv1D(8, 3, activation='relu')(inpt)
    pool2_1 = AveragePooling1D(pool_size=5)(conv2)
    conv2_1 = Conv1D(16, 3, activation='relu')(pool2_1)
    pool2_2 = GlobalMaxPooling1D()(conv2_1)
    convs.append(pool2_2)
    
    
    out = Concatenate()(convs)
    first_segment_model = Model(inputs=[inpt], outputs=[out])
    
    model = Sequential()
    model.add(first_segment_model)
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    
    print(first_segment_model.summary())
    print(model.summary())
    return model

In [ ]:
features = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'acceleration_gravity_diff_magnitude', 
            'Bearing', 'gyro_x_filtered', 'gyro_y_filtered', 'gyro_z_filtered', 'gyro_filtered_magnitude',
            'Speed', 'Accuracy', 'second', 'second_diff', 'orientation_theta', 'orientation_psi', 'orientation_phi']

train_feature_cnn = train_dataset_prep.copy()

train_feature_cnn[['acceleration_x', 'acceleration_y', 'acceleration_z']] = \
    train_feature_cnn[['acceleration_x', 'acceleration_y', 'acceleration_z']] / 10.0
train_feature_cnn['Bearing'] = train_feature_cnn['Bearing'] / 360.0
train_feature_cnn[['orientation_theta', 'orientation_psi', 'orientation_phi']] = \
    train_feature_cnn[['orientation_theta', 'orientation_psi', 'orientation_phi']] / 180.0
train_feature_cnn['Speed'] = train_feature_cnn['Speed'] / 35
train_feature_cnn['second'] = train_feature_cnn['second'] / 1750
train_feature_cnn['second_diff'] = train_feature_cnn['second_diff'] / 30
train_feature_cnn['Accuracy'] = train_feature_cnn['Accuracy'] / 15

train_feature_cnn, train_booking_ids = to_keras_input(train_feature_cnn, features, maxlen=200)

In [ ]:
model = create_model_cnn_2(train_feature_cnn)
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['binary_accuracy'])
train_seqlabel = pd.merge(train_booking_ids, train_label, on='bookingID').label


callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='binary_accuracy', patience=3)
]
history = model.fit(train_feature_cnn,
                    np.array(train_seqlabel).reshape((-1,1)),
                    batch_size=4,
                    epochs=50,
                    callbacks=callbacks_list,
                    validation_split=0.2,
                    verbose=1)

In [ ]:
test_feature_cnn = test_dataset_prep.copy()

test_feature_cnn[['acceleration_x', 'acceleration_y', 'acceleration_z']] = \
    test_feature_cnn[['acceleration_x', 'acceleration_y', 'acceleration_z']] / 10.0
test_feature_cnn[['gyro_x_filtered', 'gyro_y_filtered', 'gyro_z_filtered']] = \
    test_feature_cnn[['gyro_x_filtered', 'gyro_y_filtered', 'gyro_z_filtered']] 
test_feature_cnn['Bearing'] = test_feature_cnn['Bearing'] / 360.0
test_feature_cnn[['orientation_theta', 'orientation_psi', 'orientation_phi']] = \
    test_feature_cnn[['orientation_theta', 'orientation_psi', 'orientation_phi']] / 180.0
test_feature_cnn['Speed'] = test_feature_cnn['Speed'] / 35.0
test_feature_cnn['second'] = test_feature_cnn['second'] / 1750.0
test_feature_cnn['second_diff'] = test_feature_cnn['second_diff'] / 30.0
test_feature_cnn['Accuracy'] = test_feature_cnn['Accuracy'] / 15.0

test_feature_cnn, test_booking_ids = to_keras_input(test_feature_cnn, features, maxlen=200)

pred = model.predict_proba(test_feature_cnn)
prediction_df = pd.DataFrame(data={'bookingID': test_booking_ids.bookingID, 'prediction': pred[:,0]})
evaluate(prediction_df, test_label)

In [ ]:
plot_roc(prediction_df, test_label)

In [ ]:
sns.distplot(pd.Series(pred[:,0]))

## 4. CNN - Random Forest Aggregate (Best)

In [ ]:
cnn_model = Sequential()

for layer in model.layers[:-1]:
    cnn_model.add(layer)    

# Freeze the layers 
for layer in model.layers:
    layer.trainable = False

test_cnn_emb = cnn_model.predict_proba(test_feature_cnn)
train_cnn_emb = cnn_model.predict_proba(train_feature_cnn)

In [ ]:
agg_features = train_agg_data.columns[train_agg_data.columns.str.contains("max|std|ratio")]

train_data_stack = pd.concat([
    pd.Series(train_booking_ids.bookingID), 
    train_agg_data[agg_features],
    pd.DataFrame(train_cnn_emb, columns=['cnn_result_'+ str(i) for i in range(len(train_cnn_emb[0]))])
], axis=1)

test_data_stack = pd.concat([
    pd.Series(test_booking_ids.bookingID), 
    test_agg_data[agg_features],
    pd.DataFrame(test_cnn_emb, columns=['cnn_result_'+ str(i) for i in range(len(test_cnn_emb[0]))])
], axis=1)

train_data_stack = pd.merge(train_data_stack, train_label, on='bookingID')
train_data_stack_features = train_data_stack.columns[train_data_stack.columns != 'label']

### Stacking

In [ ]:
stack_clf = RandomForestClassifier(n_estimators=200, random_state=0, min_samples_leaf=75)
stack_clf.fit(train_data_stack[train_data_stack_features], train_data_stack.label)
pred = stack_clf.predict_proba(test_data_stack[train_data_stack_features])
pred = pred[:,np.argwhere(cls.classes_==1)[0][0]]

prediction_df = pd.DataFrame(data={'bookingID':test_data_stack.bookingID, 'prediction': pred})
print('AUC:',evaluate(prediction_df, test_label))
plot_roc(prediction_df, test_label);

evaluate(prediction_df, test_label)

In [ ]:
sns.distplot(pred)